In [2]:
import pandas as pd
import numpy as np
from Bio import SeqIO

In [4]:
file = "/home/ksuga/UTRBERT/data/gencode.v17.pc_transcripts.fa"
utr5 = []
utr3 = []
both_utr = []
for i,rec in enumerate(SeqIO.parse(file,"fasta")):
    desc = rec.description
    enstid = desc.split("|")[0]
    regions = desc.split("|")[7:-1]
    flag_5=0
    flag_3=0
    for reg in regions:
        if "UTR5" in reg:
            utr5.append(enstid)
            flag_5 = True
        elif "UTR3" in reg:
            utr3.append(enstid)
            flag_3 = True
    if flag_5 and flag_3:
        both_utr.append(enstid)
    
    


print(f"utr5:{len(utr5)}")
print(f"utr3:{len(utr3)}")
print(f"both_utr:{len(both_utr)}")

utr5:79899
utr3:71784
both_utr:61218


In [8]:
counter = 0
for i,rec in enumerate(SeqIO.parse(file,"fasta")):
    counter+=1
print(counter)

94917


In [10]:
from Bio import SeqIO
import pandas as pd

# Gencode v38 transcripts fastaファイルのパス
fasta_file = "/home/ksuga/UTRBERT/data/gencode.v17.pc_transcripts.fa"

# UTRおよびCDSの開始および終了位置を格納する空のリストを作成
UTR5_start, UTR5_end, CDS_start, CDS_end, UTR3_start, UTR3_end = [], [], [], [], [], []

# ENST IDと配列を格納する辞書を作成
seq_dict = {}

# fastaファイルを1つずつ読み込み、UTRおよびCDSの位置を取得し、辞書に配列を格納
for record in SeqIO.parse(fasta_file, "fasta"):
    enst_id = record.id.split("|")[0]  # ENST IDのみを取得
    seq = str(record.seq)
    desc = record.description
    regions = desc.split("|")[7:-1]
    flag_5=0
    flag_3=0
    for reg in regions:
        if "UTR5" in reg:
            utr5.append(enstid)
            flag_5 = True
        elif "UTR3" in reg:
            utr3.append(enstid)
            flag_3 = True
    if flag_5 and flag_3:
        UTR5, CDS, UTR3 = regions[0],regions[1],regions[2] # UTRおよびCDSの位置を取得
        UTR5_start.append(int(UTR5.split(":")[1].split("-")[0]))  # UTR5の開始位置をリストに追加
        UTR5_end.append(int(UTR5.split(":")[1].split("-")[1]))  # UTR5の終了位置をリストに追加
        CDS_start.append(int(CDS.split(":")[1].split("-")[0]))  # CDSの開始位置をリストに追加
        CDS_end.append(int(CDS.split(":")[1].split("-")[1]))  # CDSの終了位置をリストに追加
        UTR3_start.append(int(UTR3.split(":")[1].split("-")[0]))  # UTR3の開始位置をリストに追加
        UTR3_end.append(int(UTR3.split(":")[1].split("-")[1]))  # UTR3の終了位置をリストに追加
        seq_dict[enst_id] = seq

# リストを使用してpandas DataFrameを作成
df = pd.DataFrame({
    "ENST_ID": list(seq_dict.keys()),
    "fiveprime": [seq_dict[k][s-1:e-1] for k, s, e in zip(seq_dict.keys(), UTR5_start, UTR5_end)],
    "cds": [seq_dict[k][s-1:e-1] for k, s, e in zip(seq_dict.keys(), CDS_start, CDS_end)],
    "threeprime": [seq_dict[k][s-1:e-1] for k, s, e in zip(seq_dict.keys(), UTR3_start, UTR3_end)]
})



In [11]:
for reg in ["fiveprime","cds","threeprime"]:
    seqs = df[reg]
    lens = list(map(len,seqs))
    df[f"{reg}_len"] = lens
df

,ENST_ID,fiveprime,cds,threeprime,fiveprime_len,cds_len,threeprime_len
0,ENST00000423372.3,GCCTCTGCCTCCCGTCAGCCTCTACAGTCCCAACGTCTGCCTCACA...,ACCTCACTGTGGACCCCCCAAGCCAAGCTCCCAACCTTTCAGCAGC...,CTGTGAGGCCATTTCCAGGCCTAGTGCCTGCCTCGTGGCTGACTCT...,69,779,1810
1,ENST00000426406.1,AGCCCAGTTGGCTGGACC,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATGATATAATAAGCCCTTCTCATTAAACATGATATG,18,938,36
2,ENST00000332831.2,AGCCCAGTTGGCTGGACC,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATGATATAATAAGCCCTTCTCATTAAACATGATATG,18,938,36
3,ENST00000342066.3,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,ATGTCCAAGGGGATCCTGCAGGTGCATCCTCCGATCTGCGACTGCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,82,2045,421
4,ENST00000327044.6,CGCGCGGCATTCTGGGGCCGGAAGTGGGGTGCACGCTTCGGGTTGGTGT,ATGGCAGCTGCGGGGAGCCGCAAGAGGCGCCTGGCGGAGCTGACGG...,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...,49,2249,489
...,...,...,...,...,...,...,...
61213,ENST00000460621.1,AGTTGGCGACCCTGCACTGACCCGCGTCCCTCCGTCCCGAGCCCGC...,ATGGCGATTCTTTTTGCTGTTGTTGCCAGGGGGACCACTATCCTTG...,GAAAGAAGAAGTTACCATTAACCAAGGATATGAGAGA,79,539,37
61214,ENST00000244174.5,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGACTGGGCAGATGCATCTGGGAAGGCTGGACCTTGGAGAGTG...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,178,1565,198
61215,ENST00000424344.3,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGCACCTGGCTCCTGGCCTGCATCTGCATCTGCACCTGTGTCT...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,366,1502,407
61216,ENST00000369423.2,CAGAGATAGTTGGGTGACAAATCACCTCCAGGTTGGG,ATGCCTCAGACTTGTGATGGGACTGGGCAGATGCATCTGGGAAGTA...,GCCAGGACTGTGCTGGCACCCCACAGGGAGCCTTGGAGCCCTGCGT...,37,1028,964


In [12]:
truc = lambda seq: seq.split(".")[0]
df["trans_id"] = list(map(truc,df["ENST_ID"]))
df

,ENST_ID,fiveprime,cds,threeprime,fiveprime_len,cds_len,threeprime_len,trans_id
0,ENST00000423372.3,GCCTCTGCCTCCCGTCAGCCTCTACAGTCCCAACGTCTGCCTCACA...,ACCTCACTGTGGACCCCCCAAGCCAAGCTCCCAACCTTTCAGCAGC...,CTGTGAGGCCATTTCCAGGCCTAGTGCCTGCCTCGTGGCTGACTCT...,69,779,1810,ENST00000423372
1,ENST00000426406.1,AGCCCAGTTGGCTGGACC,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATGATATAATAAGCCCTTCTCATTAAACATGATATG,18,938,36,ENST00000426406
2,ENST00000332831.2,AGCCCAGTTGGCTGGACC,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATGATATAATAAGCCCTTCTCATTAAACATGATATG,18,938,36,ENST00000332831
3,ENST00000342066.3,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,ATGTCCAAGGGGATCCTGCAGGTGCATCCTCCGATCTGCGACTGCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,82,2045,421,ENST00000342066
4,ENST00000327044.6,CGCGCGGCATTCTGGGGCCGGAAGTGGGGTGCACGCTTCGGGTTGGTGT,ATGGCAGCTGCGGGGAGCCGCAAGAGGCGCCTGGCGGAGCTGACGG...,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...,49,2249,489,ENST00000327044
...,...,...,...,...,...,...,...,...
61213,ENST00000460621.1,AGTTGGCGACCCTGCACTGACCCGCGTCCCTCCGTCCCGAGCCCGC...,ATGGCGATTCTTTTTGCTGTTGTTGCCAGGGGGACCACTATCCTTG...,GAAAGAAGAAGTTACCATTAACCAAGGATATGAGAGA,79,539,37,ENST00000460621
61214,ENST00000244174.5,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGACTGGGCAGATGCATCTGGGAAGGCTGGACCTTGGAGAGTG...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,178,1565,198,ENST00000244174
61215,ENST00000424344.3,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGCACCTGGCTCCTGGCCTGCATCTGCATCTGCACCTGTGTCT...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,366,1502,407,ENST00000424344
61216,ENST00000369423.2,CAGAGATAGTTGGGTGACAAATCACCTCCAGGTTGGG,ATGCCTCAGACTTGTGATGGGACTGGGCAGATGCATCTGGGAAGTA...,GCCAGGACTGTGCTGGCACCCCACAGGGAGCCTTGGAGCCCTGCGT...,37,1028,964,ENST00000369423


In [13]:
df.to_csv("gencode_v17_full_regions.csv")

In [8]:
df[df["UTR5_len"]>=30]

,ENST_ID,UTR5,CDS,UTR3,UTR5_len,CDS_len,UTR3_len
0,ENST00000423372.3,GCCTCTGCCTCCCGTCAGCCTCTACAGTCCCAACGTCTGCCTCACA...,ACCTCACTGTGGACCCCCCAAGCCAAGCTCCCAACCTTTCAGCAGC...,CTGTGAGGCCATTTCCAGGCCTAGTGCCTGCCTCGTGGCTGACTCT...,69,779,1810
3,ENST00000342066.3,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,ATGTCCAAGGGGATCCTGCAGGTGCATCCTCCGATCTGCGACTGCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,82,2045,421
4,ENST00000327044.6,CGCGCGGCATTCTGGGGCCGGAAGTGGGGTGCACGCTTCGGGTTGGTGT,ATGGCAGCTGCGGGGAGCCGCAAGAGGCGCCTGGCGGAGCTGACGG...,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...,49,2249,489
5,ENST00000338591.3,AGTGAGCGACACAGAGCGGGCCGCCACCGCCGAGCAGCCCTCCGGC...,ATGCAGCCCCGCAGCGAGCGCCCGGCCGGCAGGACGCAGAGCCCGG...,CCCACCTACCACCAGAGGCCTGCAGCCTCCCACATGCCTTAAGGGG...,106,1928,523
6,ENST00000379410.3,GGGACCCAGACTTGCCGACCTGTACGACTCTGGC,ATGGGGAACAGCCACTGTGTCCCTCAGGCCCCCAGGAGGCTCCGGG...,TGGCCGCGGTGAGGTGGGTTCTCAGGACCACCCTCGCCAAGCTCCA...,34,1835,532
...,...,...,...,...,...,...,...
61213,ENST00000460621.1,AGTTGGCGACCCTGCACTGACCCGCGTCCCTCCGTCCCGAGCCCGC...,ATGGCGATTCTTTTTGCTGTTGTTGCCAGGGGGACCACTATCCTTG...,GAAAGAAGAAGTTACCATTAACCAAGGATATGAGAGA,79,539,37
61214,ENST00000244174.5,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGACTGGGCAGATGCATCTGGGAAGGCTGGACCTTGGAGAGTG...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,178,1565,198
61215,ENST00000424344.3,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGCACCTGGCTCCTGGCCTGCATCTGCATCTGCACCTGTGTCT...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,366,1502,407
61216,ENST00000369423.2,CAGAGATAGTTGGGTGACAAATCACCTCCAGGTTGGG,ATGCCTCAGACTTGTGATGGGACTGGGCAGATGCATCTGGGAAGTA...,GCCAGGACTGTGCTGGCACCCCACAGGGAGCCTTGGAGCCCTGCGT...,37,1028,964


In [14]:
non_ids = []
for i,cds in enumerate(df["cds"]):
    if cds[:3]!="ATG":
        non_ids.append(i)
print(len(non_ids))

80


In [20]:
df_rest = df.drop(non_ids)
df_rest

,ENST_ID,fiveprime,cds,threeprime,fiveprime_len,cds_len,threeprime_len,trans_id
1,ENST00000426406.1,AGCCCAGTTGGCTGGACC,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATGATATAATAAGCCCTTCTCATTAAACATGATATG,18,938,36,ENST00000426406
2,ENST00000332831.2,AGCCCAGTTGGCTGGACC,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATGATATAATAAGCCCTTCTCATTAAACATGATATG,18,938,36,ENST00000332831
3,ENST00000342066.3,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,ATGTCCAAGGGGATCCTGCAGGTGCATCCTCCGATCTGCGACTGCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,82,2045,421,ENST00000342066
4,ENST00000327044.6,CGCGCGGCATTCTGGGGCCGGAAGTGGGGTGCACGCTTCGGGTTGGTGT,ATGGCAGCTGCGGGGAGCCGCAAGAGGCGCCTGGCGGAGCTGACGG...,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...,49,2249,489,ENST00000327044
5,ENST00000338591.3,AGTGAGCGACACAGAGCGGGCCGCCACCGCCGAGCAGCCCTCCGGC...,ATGCAGCCCCGCAGCGAGCGCCCGGCCGGCAGGACGCAGAGCCCGG...,CCCACCTACCACCAGAGGCCTGCAGCCTCCCACATGCCTTAAGGGG...,106,1928,523,ENST00000338591
...,...,...,...,...,...,...,...,...
61213,ENST00000460621.1,AGTTGGCGACCCTGCACTGACCCGCGTCCCTCCGTCCCGAGCCCGC...,ATGGCGATTCTTTTTGCTGTTGTTGCCAGGGGGACCACTATCCTTG...,GAAAGAAGAAGTTACCATTAACCAAGGATATGAGAGA,79,539,37,ENST00000460621
61214,ENST00000244174.5,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGACTGGGCAGATGCATCTGGGAAGGCTGGACCTTGGAGAGTG...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,178,1565,198,ENST00000244174
61215,ENST00000424344.3,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGCACCTGGCTCCTGGCCTGCATCTGCATCTGCACCTGTGTCT...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,366,1502,407,ENST00000424344
61216,ENST00000369423.2,CAGAGATAGTTGGGTGACAAATCACCTCCAGGTTGGG,ATGCCTCAGACTTGTGATGGGACTGGGCAGATGCATCTGGGAAGTA...,GCCAGGACTGTGCTGGCACCCCACAGGGAGCCTTGGAGCCCTGCGT...,37,1028,964,ENST00000369423


In [22]:
df_rest= df_rest[df_rest["fiveprime_len"]>=30]
df_rest

,ENST_ID,fiveprime,cds,threeprime,fiveprime_len,cds_len,threeprime_len,trans_id
3,ENST00000342066.3,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,ATGTCCAAGGGGATCCTGCAGGTGCATCCTCCGATCTGCGACTGCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,82,2045,421,ENST00000342066
4,ENST00000327044.6,CGCGCGGCATTCTGGGGCCGGAAGTGGGGTGCACGCTTCGGGTTGGTGT,ATGGCAGCTGCGGGGAGCCGCAAGAGGCGCCTGGCGGAGCTGACGG...,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...,49,2249,489,ENST00000327044
5,ENST00000338591.3,AGTGAGCGACACAGAGCGGGCCGCCACCGCCGAGCAGCCCTCCGGC...,ATGCAGCCCCGCAGCGAGCGCCCGGCCGGCAGGACGCAGAGCCCGG...,CCCACCTACCACCAGAGGCCTGCAGCCTCCCACATGCCTTAAGGGG...,106,1928,523,ENST00000338591
6,ENST00000379410.3,GGGACCCAGACTTGCCGACCTGTACGACTCTGGC,ATGGGGAACAGCCACTGTGTCCCTCAGGCCCCCAGGAGGCTCCGGG...,TGGCCGCGGTGAGGTGGGTTCTCAGGACCACCCTCGCCAAGCTCCA...,34,1835,532,ENST00000379410
7,ENST00000379407.2,GGGACCCAGACTTGCCGACCTGTACGACTCTGGC,ATGGGGAACAGCCACTGTGTCCCTCAGGCCCCCAGGAGGCTCCGGG...,TGGCCGCGGTGAGGTGGGTTCTCAGGACCACCCTCGCCAAGCTCCA...,34,1730,520,ENST00000379407
...,...,...,...,...,...,...,...,...
61213,ENST00000460621.1,AGTTGGCGACCCTGCACTGACCCGCGTCCCTCCGTCCCGAGCCCGC...,ATGGCGATTCTTTTTGCTGTTGTTGCCAGGGGGACCACTATCCTTG...,GAAAGAAGAAGTTACCATTAACCAAGGATATGAGAGA,79,539,37,ENST00000460621
61214,ENST00000244174.5,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGACTGGGCAGATGCATCTGGGAAGGCTGGACCTTGGAGAGTG...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,178,1565,198,ENST00000244174
61215,ENST00000424344.3,AGCAGCTCTGTAATGCGCTTGTGGTTTCAGATGTGGGCGGCCTGTG...,ATGGGCACCTGGCTCCTGGCCTGCATCTGCATCTGCACCTGTGTCT...,GTCCCTGACTCGCCAGATGCATCATGTCCATTTTGGGAAAATGGAC...,366,1502,407,ENST00000424344
61216,ENST00000369423.2,CAGAGATAGTTGGGTGACAAATCACCTCCAGGTTGGG,ATGCCTCAGACTTGTGATGGGACTGGGCAGATGCATCTGGGAAGTA...,GCCAGGACTGTGCTGGCACCCCACAGGGAGCCTTGGAGCCCTGCGT...,37,1028,964,ENST00000369423


In [23]:
df_rest.to_csv("gencode_v17_full_regions_restricted.csv")